In [4]:
import pandas as pd

In [5]:
from module import filter_data

raw_data = "ZooSpecPhotoDR19_torradeflot.csv"
df = pd.read_csv(raw_data)

df_filtered = filter_data(df, 0.9)
df_filtered = df_filtered.reset_index()

In [6]:
from module import get_image
import os
from PIL import Image
from concurrent.futures import ThreadPoolExecutor, as_completed

photodir = "galaxy_images2"
if not os.path.exists(photodir):
    os.mkdir(photodir)


def get_galaxy(gal):
    pack = int(gal / 1000)
    packdir = os.path.join(photodir, f"pack{pack}")
    if not os.path.exists(packdir):
        os.mkdir(packdir)
    result = get_image(
        df_filtered.loc[gal, "ra"],
        df_filtered.loc[gal, "dec"],
        df_filtered.loc[gal, "petroR90_r"],
        64,
        64,
    )
    im = Image.fromarray(result)
    im = im.save(os.path.join(packdir, f"g{gal}.jpg"))
    return f"Saved g{gal}"


MAX_WORKERS = 16  # You can adjust based on CPU/network
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(get_galaxy, gal) for gal in range(min(len(df_filtered), 70000))]

    for future in as_completed(futures):
        print(future.result())

KeyboardInterrupt: 